# Heavy Tail（长尾）与鲁棒处理：clip / Huber / Quantile / log1p / Box-Cox / Yeo-Johnson

这个 notebook 用**可运行的最小示例**演示：
1. 为什么 heavy tail 会让 **MSE/RMSE 被少数极端点主导**；
2. 线性回归在离群点/长尾下为什么会不稳；
3. 常见应对：**winsorize/clip、Huber、Quantile、log1p、Box-Cox、Yeo-Johnson**；
4. 面试里建议同时汇报的指标与分桶验证方式。

> 依赖：numpy, pandas, matplotlib, scikit-learn（常见面试环境都会有）

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, HuberRegressor, QuantileRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn.preprocessing import PowerTransformer

np.random.seed(42)

## 1) 构造一个“主体分布正常 + 少量极端值”的回归数据

我们用一个简单的线性关系：
\[
y = 3x + \epsilon
\]
其中大多数噪声是正态，但会混入极少数极端噪声（重尾）。

In [ ]:
n = 5000
x = np.random.normal(size=n)

# 主体噪声：正态
eps_main = np.random.normal(scale=1.0, size=n)

# 极端噪声：少量点特别大（例如 t 分布/或直接乘大系数）
is_tail = np.random.rand(n) < 0.02  # 2% 极端点
eps_tail = np.random.normal(scale=25.0, size=n)  # 极端波动

eps = np.where(is_tail, eps_tail, eps_main)
y = 3.0 * x + eps

X = x.reshape(-1, 1)

df = pd.DataFrame({"x": x, "y": y, "is_tail": is_tail})
df.head()

### 看一下分布与极端点比例

In [ ]:
fig = plt.figure()
plt.hist(y, bins=80)
plt.title("Distribution of y (heavy-tailed)")
plt.xlabel("y")
plt.ylabel("count")
plt.show()

tail_share = df["is_tail"].mean()
tail_share

## 2) 为什么 MSE 会被少数点主导？（数值演示）

我们先训练一个普通最小二乘线性回归（OLS），然后把误差贡献按样本排序，看看 top 1% 样本贡献了多少 MSE。

In [ ]:
X_train, X_test, y_train, y_test, tail_train, tail_test = train_test_split(
    X, y, is_tail, test_size=0.3, random_state=0
)

ols = LinearRegression().fit(X_train, y_train)
pred = ols.predict(X_test)

resid = y_test - pred
sq = resid**2

mse = sq.mean()
rmse = math.sqrt(mse)
mae = np.abs(resid).mean()
medae = np.median(np.abs(resid))

mse, rmse, mae, medae

In [ ]:
# MSE 贡献占比：按平方误差从大到小排序
sq_sorted = np.sort(sq)[::-1]
cum = np.cumsum(sq_sorted) / np.sum(sq_sorted)

def share_of_top(p):
    k = int(len(sq_sorted) * p)
    return float(cum[k-1])

for p in [0.001, 0.005, 0.01, 0.05]:
    print(f"Top {p*100:.1f}% samples contribute {share_of_top(p)*100:.1f}% of total squared error.")

你会看到：**很少数的点贡献了绝大部分的平方误差**。
这就是 heavy tail 下“优化 MSE = 追着极端点跑”的根源。

## 3) 线性模型为什么不稳：对极端点敏感

我们对比：
- OLS（最小二乘）
- HuberRegressor（鲁棒损失）
- QuantileRegressor（分位数回归，默认 τ=0.5 对应中位数回归）

In [ ]:
models = {
    "OLS (MSE)": LinearRegression(),
    "Huber": HuberRegressor(epsilon=1.35),
    "Quantile (tau=0.5)": QuantileRegressor(quantile=0.5, alpha=0.0, solver="highs"),
}

results = []
for name, m in models.items():
    m.fit(X_train, y_train)
    p = m.predict(X_test)
    results.append({
        "model": name,
        "RMSE": mean_squared_error(y_test, p, squared=False),
        "MAE": mean_absolute_error(y_test, p),
        "MedAE": median_absolute_error(y_test, p),
        "coef": float(m.coef_[0]) if hasattr(m, "coef_") else np.nan,
        "intercept": float(m.intercept_) if hasattr(m, "intercept_") else np.nan,
    })

pd.DataFrame(results).sort_values("RMSE")

### 分桶看误差：主体 vs 尾部
把测试集按是否“极端点”分组，看看误差差异。

In [ ]:
def metrics(y_true, y_pred):
    return {
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
        "n": len(y_true),
    }

bucket_rows = []
for name, m in models.items():
    p = m.predict(X_test)
    bucket_rows.append({"model": name, "bucket": "ALL", **metrics(y_test, p)})
    bucket_rows.append({"model": name, "bucket": "TAIL", **metrics(y_test[tail_test], p[tail_test])})
    bucket_rows.append({"model": name, "bucket": "NON-TAIL", **metrics(y_test[~tail_test], p[~tail_test])})

pd.DataFrame(bucket_rows)

## 4) winsorize / clip（截尾/压尾）

做法：把 y（或某些特征）超过分位数阈值的部分压到阈值上。
这里演示在训练集上对 y 做 winsorize，然后再拟合 OLS（只示意，实际你也可对特征做）。

In [ ]:
def winsorize_by_quantile(arr, q_low=0.005, q_high=0.995):
    lo, hi = np.quantile(arr, [q_low, q_high])
    return np.clip(arr, lo, hi), lo, hi

y_train_clip, lo, hi = winsorize_by_quantile(y_train, 0.005, 0.995)

ols_clip = LinearRegression().fit(X_train, y_train_clip)
p_clip = ols_clip.predict(X_test)

pd.DataFrame([
    {"model": "OLS raw y", **metrics(y_test, ols.predict(X_test))},
    {"model": "OLS winsorized y_train", **metrics(y_test, p_clip)},
])

## 5) log1p / Box-Cox / Yeo-Johnson：分布变换压长尾

### 5.1 log1p（适用于 y ≥ 0）
- 变换：`z = log(1 + y)`
- 预测后还原：`y_hat = exp(z_hat) - 1`

注意：如果 y 有负值，log1p 需要先 shift 或改用 Yeo-Johnson。

In [ ]:
# 为了演示 log1p，这里构造一个非负目标（常见如成交量/负载/金额）
y_pos = np.exp(3.0 * x + np.random.normal(scale=0.8, size=n))  # 正值且长尾
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y_pos, test_size=0.3, random_state=0)

# baseline：直接回归原尺度
ols2 = LinearRegression().fit(X_train2, y_train2)
p2 = ols2.predict(X_test2)

# log1p：在 log 空间回归
z_train = np.log1p(y_train2)
ols_log = LinearRegression().fit(X_train2, z_train)
z_pred = ols_log.predict(X_test2)
p_log = np.expm1(z_pred)

pd.DataFrame([
    {"model": "OLS on y", **metrics(y_test2, p2)},
    {"model": "OLS on log1p(y) then invert", **metrics(y_test2, p_log)},
])

### 5.2 Box-Cox（要求严格 y > 0）与 Yeo-Johnson（允许 y≤0）

sklearn 的 `PowerTransformer` 支持：
- `method="box-cox"`：需要 y > 0
- `method="yeo-johnson"`：y 可以为任意实数

下面展示对**回归目标 y**做变换（也可对特征做）。

In [ ]:
# 对上面的 y_pos (严格>0) 做 Box-Cox
pt_bc = PowerTransformer(method="box-cox", standardize=True)
y_bc_train = pt_bc.fit_transform(y_train2.reshape(-1, 1)).ravel()

m_bc = LinearRegression().fit(X_train2, y_bc_train)
y_bc_pred = pt_bc.inverse_transform(m_bc.predict(X_test2).reshape(-1, 1)).ravel()

# 对原来的 y（含负值）做 Yeo-Johnson
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, y, test_size=0.3, random_state=0)
pt_yj = PowerTransformer(method="yeo-johnson", standardize=True)
y_yj_train = pt_yj.fit_transform(y_train3.reshape(-1, 1)).ravel()

m_yj = LinearRegression().fit(X_train3, y_yj_train)
y_yj_pred = pt_yj.inverse_transform(m_yj.predict(X_test3).reshape(-1, 1)).ravel()

pd.DataFrame([
    {"model": "Box-Cox on y_pos then invert", **metrics(y_test2, y_bc_pred)},
    {"model": "Yeo-Johnson on y then invert", **metrics(y_test3, y_yj_pred)},
])

## 6) 面试里建议的“注意事项清单”

- **先定位 heavy tail 在哪**：是 y 还是 X？两者处理方式不同。
- 不要只报一个指标：至少 `RMSE + MAE（或 MedAE）`，并**按分位数/桶**报告误差。
- heavy tail 下用 MSE 训练要小心：可试 `Huber / Quantile / MAE` 或做 `clip/transform`。
- `log1p` 只适用于 **y≥0**；`Box-Cox` 需要 **y>0**；`Yeo-Johnson` 可处理 **负值**。
- 变换后记得可逆：预测要 **inverse_transform** 回原尺度再汇报业务指标。